In [29]:
import pandas as pd
import numpy as np
import math
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
kp = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KenPom Team Metrics.csv')
#kp.head(10)
kp_conf_rat = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KP_Conf_Ratings.csv')
kp_conf_rat = kp_conf_rat[['Year_Conf', 'Conf_Abb', 'Conf_Rk', 'Conf_Full', 'Conf_Rating']]

d = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv')
d = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = df.replace(to_replace='NO IDEA', value=0)
df = df.replace(to_replace='---', value=0)
df = df.fillna(0)
df = df.drop(df.loc[((df['games']>39) & (df['year']==2023)) | ((df['games_n1']>39) & (df['year_n1']==2023)) | 
   ((df['games']>40) & (df['year']==2022)) | ((df['games_n1']>40) & (df['year_n1']==2022)) | 
   ((df['games']>33) & (df['year']==2021)) | ((df['games_n1']>33) & (df['year_n1']==2021)) | 
   ((df['games']>35) & (df['year']==2020)) | ((df['games_n1']>35) & (df['year_n1']==2020)) | 
   ((df['games']>40) & (df['year']==2019)) | ((df['games_n1']>40) & (df['year_n1']==2019)) |
   (df['year']==0)]['Player.Name'].index)
# Add Year in School KP
year_cond = [(df['Yr_n1']=='Fr'), (df['Yr_n1']=='So'), 
             (df['Yr_n1']=='Jr'), (df['Yr_n1']=='Sr')]
year_val = [1, 2, 3, 4]
df['Class_n1'] = np.select(year_cond, year_val)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['X.Min']*(df['ws']-min(df['ws']))
df['Calc A2'] = df['X.Min']*df['bpm']
# df['Calc A2'] = df['bpm']/(1-df['X.Min'])
df['Calc A3'] = df['X.Min']*df['ORtg']
df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
# df['Calc A2_n1'] = df['bpm_n1']/(1-df['X.Min_n1'])
df['Calc A_n1'] =df['X.Min_n1']*(df['ws_n1']-min(df['ws_n1']))
df['Calc A3_n1'] = df['X.Min_n1']*df['ORtg_n1']
df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR.'])*df['X.Min']
df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['X.Min']
df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
df['Calc Overall A'] = df['Calc A'] + df['Calc B']
df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
df['Calc Overall A_n1'] = df['Calc A_n1'] + df['Calc B_n1']
df['Calc Overall A2_n1'] = df['Calc A2_n1'] + df['Calc B_n1']
df['Calc Overall A3_n1'] = df['Calc A3_n1'] + df['Calc B_n1']

df = df[['Player.Name', 'player_type', 'season_n1', 'Team_n1', 'conf_abbr_n1', 'Year_dif', 'Team_AdjEM_dif', 'Class_n1',
    'KP_Conf_Dif', 'Calc A_n1', 'Calc A2_n1', 'Calc A3_n1', 'Calc B_n1', 'Calc C_n1', 'Calc Overall A_n1', 'Calc Overall A2_n1', 'Calc Overall A3_n1']]
df


/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_26935/2344370804.py:6: DtypeWarning: Columns (13,16,38,144,147,169) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv')


,Player.Name,player_type,season_n1,Team_n1,conf_abbr_n1,Year_dif,Team_AdjEM_dif,Class_n1,KP_Conf_Dif,Calc A_n1,Calc A2_n1,Calc A3_n1,Calc B_n1,Calc C_n1,Calc Overall A_n1,Calc Overall A2_n1,Calc Overall A3_n1
0,aamahne-santos-1,1,2019-20,JACKSONVILLE,A-Sun,1.0,0.000000,4,0.00,170.00,-258.40,6215.20,618.80,-516.80,788.80,360.40,6834.00
1,aamir-simms-1,1,2019-20,CLEMSON,ACC,1.0,0.000000,3,0.00,385.05,596.45,8093.60,1676.10,1155.15,2061.15,2272.55,9769.70
2,aamir-simms-1,1,2020-21,CLEMSON,ACC,1.0,0.000000,4,0.00,339.48,642.06,8324.64,1453.86,1461.24,1793.34,2095.92,9778.50
3,aanen-moody-1,1,2019-20,NORTH DAKOTA,Summit,1.0,0.000000,2,0.00,11.05,-24.65,742.90,112.20,-17.85,123.25,87.55,855.10
4,aanen-moody-1,0,2020-21,SOUTHERN UTAH,Big Sky,1.0,6.190432,3,1.27,52.63,5.54,3360.01,337.94,390.57,390.57,343.48,3697.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,zoar-nedd-1,1,2021-22,HAWAII,Big West,2.0,0.000000,2,0.00,15.60,-50.40,1006.80,195.60,-63.60,211.20,145.20,1202.40
11036,zoar-nedd-1,1,2022-23,HAWAII,Big West,1.0,0.000000,3,0.00,3.20,-21.12,273.92,25.60,-61.76,28.80,4.48,299.52
11039,zyon-pullin-1,1,2020-21,UC-RIVERSIDE,Big West,1.0,0.000000,2,0.00,316.52,285.64,8677.28,1358.72,1351.00,1675.24,1644.36,10036.00
11040,zyon-pullin-1,1,2021-22,UC-RIVERSIDE,Big West,1.0,0.000000,3,0.00,418.50,209.25,9006.12,1581.93,1205.28,2000.43,1791.18,10588.05


In [31]:
br_pos = pd.read_excel("/Users/caldavis/Documents/GitHub/Capstone-project/Player ID MASTER (2018-19 - 2022-23).xlsx")
br_pos = br_pos[['PLAYER_ID', 'Season', 'Pos']]
br_pos

,PLAYER_ID,Season,Pos
0,chris-clemons-2,2018-19,G
1,justin-wright-foreman-1,2018-19,G
2,antoine-davis-2,2022-23,G
3,carsen-edwards-1,2018-19,G
4,rj-barrett-1,2018-19,F
...,...,...,...
24794,eric-zalewski-1,2019-20,G
24795,eric-zalewski-1,2020-21,G
24796,ryan-zambie-1,2020-21,G
24797,ryan-zambie-1,2021-22,G


In [43]:
a = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")
a = a[['Player Name', 'season', 'Team', 'conf_abbr', 'games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots']]
a[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots']].apply(pd.to_numeric, errors="ignore")
a = a.replace(to_replace='NO IDEA', value=0)
a = a.replace(to_replace='---', value=0)
a = a.fillna(0)
a = a.drop(a.loc[((a['games']>39) & (a['year']==2023)) | ((a['games']>40) & (a['year']==2022)) | 
   ((a['games']>33) & (a['year']==2021)) |  
   ((a['games']>35) & (a['year']==2020)) | 
   ((a['games']>40) & (a['year']==2019)) |
   (a['year']==0)]['Player Name'].index)


/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_26935/1579229443.py:1: DtypeWarning: Columns (10,13,16,19,35) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")


In [34]:
df2 = pd.merge(df, br_pos, left_on=['Player.Name', 'season_n1'], right_on=['PLAYER_ID', 'Season'], how='left')
df2 = df2[['PLAYER_ID', 'Season', 'Pos', 'player_type', 'Team_n1', 'conf_abbr_n1',
       'Year_dif', 'Team_AdjEM_dif', 'Class_n1', 'KP_Conf_Dif', 'Calc A_n1',
       'Calc A2_n1', 'Calc A3_n1', 'Calc B_n1', 'Calc C_n1',
       'Calc Overall A_n1', 'Calc Overall A2_n1', 'Calc Overall A3_n1']]
df2
#df.columns


,PLAYER_ID,Season,Pos,player_type,Team_n1,conf_abbr_n1,Year_dif,Team_AdjEM_dif,Class_n1,KP_Conf_Dif,Calc A_n1,Calc A2_n1,Calc A3_n1,Calc B_n1,Calc C_n1,Calc Overall A_n1,Calc Overall A2_n1,Calc Overall A3_n1
0,aamahne-santos-1,2019-20,G,1,JACKSONVILLE,A-Sun,1.0,0.000000,4,0.00,170.00,-258.40,6215.20,618.80,-516.80,788.80,360.40,6834.00
1,aamir-simms-1,2019-20,F,1,CLEMSON,ACC,1.0,0.000000,3,0.00,385.05,596.45,8093.60,1676.10,1155.15,2061.15,2272.55,9769.70
2,aamir-simms-1,2020-21,F,1,CLEMSON,ACC,1.0,0.000000,4,0.00,339.48,642.06,8324.64,1453.86,1461.24,1793.34,2095.92,9778.50
3,aanen-moody-1,2019-20,G,1,NORTH DAKOTA,Summit,1.0,0.000000,2,0.00,11.05,-24.65,742.90,112.20,-17.85,123.25,87.55,855.10
4,aanen-moody-1,2020-21,G,0,SOUTHERN UTAH,Big Sky,1.0,6.190432,3,1.27,52.63,5.54,3360.01,337.94,390.57,390.57,343.48,3697.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9457,zoar-nedd-1,2021-22,F,1,HAWAII,Big West,2.0,0.000000,2,0.00,15.60,-50.40,1006.80,195.60,-63.60,211.20,145.20,1202.40
9458,zoar-nedd-1,2022-23,F,1,HAWAII,Big West,1.0,0.000000,3,0.00,3.20,-21.12,273.92,25.60,-61.76,28.80,4.48,299.52
9459,zyon-pullin-1,2020-21,G,1,UC-RIVERSIDE,Big West,1.0,0.000000,2,0.00,316.52,285.64,8677.28,1358.72,1351.00,1675.24,1644.36,10036.00
9460,zyon-pullin-1,2021-22,G,1,UC-RIVERSIDE,Big West,1.0,0.000000,3,0.00,418.50,209.25,9006.12,1581.93,1205.28,2000.43,1791.18,10588.05


In [61]:
df_final = pd.merge(df2, a, left_on=['PLAYER_ID', 'Season'], right_on=['Player Name', 'season'], how='inner')
pd.set_option('display.max_columns', None)
df_final['fg_pct']=df_final['fg_pct'].astype(str).astype(float)
df_final['fg2_pct']=df_final['fg2_pct'].astype(str).astype(float)
df_final['fg3_pct']=df_final['fg3_pct'].astype(str).astype(float)
df_final['ft_pct']=df_final['ft_pct'].astype(str).astype(float)
df_final = df_final[(df_final['Season']=='2022-23') | (df_final['Season']=='2021-22')]
df_final = df_final[['PLAYER_ID', 'Season', 'Pos', 'player_type', 'Calc A2_n1', 'ORtg', 'KP_Conf_Dif', 'orb_pct', 
                     'Calc A3_n1', 'Calc B_n1', 'bpm', 'tov_pct', 'efg_pct', 'ws', 'Year_dif']]
df_final

,PLAYER_ID,Season,Pos,player_type,Calc A2_n1,ORtg,KP_Conf_Dif,orb_pct,Calc A3_n1,Calc B_n1,bpm,tov_pct,efg_pct,ws,Year_dif
5,aanen-moody-1,2021-22,G,1,42.57,124.0,0.00,1.4,5865.20,648.01,0.9,8.4,0.591,2.4,1.0
6,aanen-moody-1,2022-23,G,0,131.24,115.4,0.00,1.6,8908.88,1003.60,1.7,10.5,0.562,3.3,1.0
8,aaron-cash-1,2022-23,G,0,-44.40,96.8,-15.97,7.7,5372.40,999.00,-0.8,17.7,0.549,1.7,1.0
11,aaron-clarke-1,2021-22,G,1,-226.24,110.5,0.00,1.2,7812.35,374.71,-3.2,14.1,0.488,2.3,1.0
12,aaron-clarke-1,2022-23,G,0,-45.43,83.7,9.96,0.0,493.83,83.78,-7.7,18.5,0.371,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9465,zion-young-1,2021-22,G,1,-19.14,105.3,0.00,1.5,694.98,93.72,-2.9,7.9,0.500,0.2,1.0
9467,zoar-nedd-1,2021-22,F,1,-50.40,83.9,0.00,2.6,1006.80,195.60,-4.2,12.2,0.400,0.3,2.0
9468,zoar-nedd-1,2022-23,F,1,-21.12,85.6,0.00,9.9,273.92,25.60,-6.6,6.7,0.250,0.0,1.0
9470,zyon-pullin-1,2021-22,G,1,209.25,107.6,0.00,1.3,9006.12,1581.93,2.5,15.3,0.498,4.0,1.0


In [66]:
# df_simp = df[['fg_pct', 'fg2_pct', 'fg3_pct', 'ft_pct', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
#     'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 
#     'Ht', 'Wt', 'Conf_Rating', 'sos']]

df_simp = df_final[['player_type', 'Calc A2_n1', 'ORtg', 'KP_Conf_Dif', 'orb_pct', 'Calc A3_n1', 'Calc B_n1', 'bpm', 'tov_pct', 'efg_pct', 'ws', 'Year_dif']]
#df_simp = list(set(df_simp))

df_std = (df_simp-df_simp.mean())/df_simp.std()

df_name = df_final[['PLAYER_ID', 'Season', 'Pos']]
df_name['year'] = df_name['Season'].str[-2:].astype(int)+2000
df_name['Player_Year'] = df_name['PLAYER_ID'] + "_" + df_name['year'].astype(int).astype(str)
df_std = pd.merge(df_name, df_std, left_index=True, right_index=True)
df_std = df_std.drop(['PLAYER_ID', 'year', 'Season'], axis=1)

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_26935/1930930861.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_name['year'] = df_name['Season'].str[-2:].astype(int)+2000
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_26935/1930930861.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_name['Player_Year'] = df_name['PLAYER_ID'] + "_" + df_name['year'].astype(int).astype(str)


In [67]:
df_std

,Pos,Player_Year,player_type,Calc A2_n1,ORtg,KP_Conf_Dif,orb_pct,Calc A3_n1,Calc B_n1,bpm,tov_pct,efg_pct,ws,Year_dif
5,G,aanen-moody-1_2022,0.629132,0.081123,1.600778,0.057897,-1.002075,0.347153,-0.256663,0.329628,-1.208621,0.993091,0.438969,0.126888
6,G,aanen-moody-1_2023,-1.589179,0.499802,1.010789,0.057897,-0.950709,1.444126,0.446459,0.508030,-0.874570,0.666803,1.057318,0.126888
8,G,aaron-cash-1_2023,-1.589179,-0.329530,-0.265233,-2.575236,0.615952,0.169543,0.437363,-0.049474,0.270747,0.520536,-0.041969,0.126888
11,G,aaron-clarke-1_2022,0.629132,-1.188137,0.674633,0.057897,-1.053441,1.048926,-0.797070,-0.584678,-0.301911,-0.165793,0.370264,0.126888
12,G,aaron-clarke-1_2023,-1.589179,-0.334393,-1.163936,1.700101,-1.361637,-1.588744,-1.372337,-1.588185,0.398005,-1.482195,-1.209960,0.126888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9465,G,zion-young-1_2022,0.629132,-0.210258,0.317896,0.057897,-0.976392,-1.516247,-1.352683,-0.517777,-1.288157,-0.030777,-1.072550,0.126888
9467,F,zoar-nedd-1_2022,0.629132,-0.357861,-1.150215,0.057897,-0.693880,-1.403864,-1.151231,-0.807679,-0.604148,-1.155907,-1.003844,0.130838
9468,F,zoar-nedd-1_2023,0.629132,-0.219607,-1.033590,0.057897,1.180978,-1.668002,-1.487379,-1.342883,-1.479043,-2.843602,-1.209960,0.126888
9470,G,zyon-pullin-1_2022,0.629132,0.868148,0.475683,0.057897,-1.027758,1.479173,1.590013,0.686431,-0.111025,-0.053280,1.538256,0.126888


In [71]:
df_g = df_std[df_std['Pos']=='G']
df_g = df_g.drop(['Pos'], axis=1)
df_g_col = list(df_g['Player_Year']) # List of player-seasons
df_g = df_g.set_index('Player_Year') # Setting index to player-season

df_g_cs = cosine_similarity(df_g, df_g)
df_g_cs= pd.DataFrame(df_g_cs, columns=df_g_col)
df_g_cs['Player'] = df_g_col
df_g_cs = df_g_cs.set_index('Player')




In [ ]:
df_f = df_std[df_std['Pos']=='F']
df_f = df_f.drop(['Pos'], axis=1)
df_f_col = list(df_f['Player_Year']) # List of player-seasons
df_f = df_f.set_index('Player_Year') # Setting index to player-season

df_f_cs = cosine_similarity(df_f, df_f)
df_f_cs= pd.DataFrame(df_f_cs, columns=df_f_col)
df_f_cs['Player'] = df_f_col
df_f_cs = df_f_cs.set_index('Player')

In [72]:

df_c = df_std[df_std['Pos']=='C']
df_c = df_c.drop(['Pos'], axis=1)
df_c_col = list(df_c['Player_Year']) # List of player-seasons
df_c = df_c.set_index('Player_Year') # Setting index to player-season

df_c_cs = cosine_similarity(df_c, df_c)
df_c_cs= pd.DataFrame(df_c_cs, columns=df_c_col)
df_c_cs['Player'] = df_c_col
df_c_cs = df_c_cs.set_index('Player')

In [73]:
df_c_cs

,abba-lawal-1_2022,adam-cisse-1_2023,ahsan-asadullah-1_2022,ahsan-asadullah-1_2023,aj-braun-1_2023,alex-christie-1_2022,alex-tew-1_2023,alex-timmerman-1_2022,alex-timmerman-1_2023,amari-mccray-1_2022,andre-screen-1_2022,andre-screen-1_2023,anton-jansson-1_2022,anton-jansson-1_2023,ari-boya-1_2022,aziz-bandaogo-1_2023,ben-hendriks-1_2023,benjamin-griscti-1_2023,bent-leuchten-1_2023,blake-huggins-1_2022,branden-carlson-1_2022,branden-carlson-1_2023,brandon-stone-1_2022,brandon-stone-1_2023,braxton-meah-1_2022,braxton-meah-1_2023,brock-bertram-1_2022,buay-koka-1_2022,callum-mcrae-1_2022,callum-mcrae-1_2023,carson-basham-1_2023,charles-bediako-1_2023,charles-coleman-1_2022,charles-coleman-1_2023,chol-marial-1_2023,chris-efretuei-1_2022,chris-sodom-1_2022,chris-vogt-1_2022,christian-koloko-1_2022,churchill-bounds-1_2022,churchill-bounds-1_2023,clifford-omoruyi-1_2022,clifford-omoruyi-1_2023,corbin-merritt-1_2022,dajour-dickens-1_2022,dallas-james-1_2023,david-appelgren-1_2022,david-muoka-1_2022,david-muoka-1_2023,dawson-carper-1_2022,dawson-carper-1_2023,deshawn-thomas-1_2022,dishon-jackson-1_2022,dj-nussbaum-1_2022,dj-nussbaum-1_2023,dylan-brougham-1_2022,dylan-brougham-1_2023,dylan-cardwell-1_2022,dylan-cardwell-1_2023,efe-abogidi-1_2022,efton-reid-1_2023,ej-williams-1_2022,ej-williams-1_2023,emmanuel-dowuona-1_2022,emmanuel-dowuona-1_2023,emmanuel-okpomo-1_2022,emmanuel-okpomo-1_2023,emmanuel-tshimanga-1_2022,emmanuel-tshimanga-1_2023,enoch-boakye-1_2023,essam-mostafa-1_2022,essam-mostafa-1_2023,even-brauns-1_2022,even-brauns-1_2023,francisco-caffaro-1_2022,francisco-caffaro-1_2023,franck-kepnang-1_2022,franck-kepnang-1_2023,frank-anselem-1_2022,frank-anselem-1_2023,gabe-wiznitzer-1_2022,gabe-wiznitzer-1_2023,goran-miladinovic-1_2022,goran-miladinovic-1_2023,grant-asman-1_2023,harry-morrice-1_2022,hayden-koval-1_2022,hugo-clarkin-1_2022,hunter-dickinson-1_2022,hunter-dickinson-1_2023,jack-wilson-1_2023,jackson-ames-1_2022,jackson-stormo-1_2022,jackson-stormo-1_2023,jacob-germany-1_2022,jacob-germany-1_2023,jacob-hutson-1_2022,jacob-hutson-1_2023,jaden-bediako-1_2022,jaden-bediako-1_2023,jake-stephens-1_2022,jake-stephens-1_2023,james-karnik-1_2022,jason-jitoboh-1_2022,jason-jitoboh-1_2023,jason-spurgin-1_2022,jason-spurgin-1_2023,jeff-otchere-1_2022,jeff-woodward-1_2022,jeff-woodward-1_2023,jeffrey-yan-2_2022,jermaine-hamlin-1_2022,jermaine-hamlin-1_2023,jesse-edwards-1_2022,jesse-edwards-1_2023,jhaylon-martinez-1_2023,joe-hedstrom-1_2023,joey-stpierre-1_2023,jonas-munson-1_2023,jonathan-kurtas-1_2023,josh-gray-2_2022,josh-gray-2_2023,joshua-morgan-1_2022,joshua-morgan-1_2023,jt-tan-1_2023,kendal-coleman-1_2022,kendal-coleman-1_2023,kevin-samuel-3_2022,kevin-samuel-3_2023,kirk-parker-1_2022,kiyron-powell-1_2023,kofi-cockburn-1_2022,kuba-karwowski-1_2022,kyler-filewich-1_2022,kyler-filewich-1_2023,larry-owens-2_2022,lawson-lovering-1_2023,ludgy-debaut-1_2022,ludgy-debaut-1_2023,lynn-kidd-1_2023,mady-sissoko-1_2022,mady-sissoko-1_2023,malcolm-dandridge-1_2022,malcolm-dandridge-1_2023,malcolm-wilson-1_2022,malcolm-wilson-1_2023,malik-williams-1_2022,mark-williams-7_2022,maros-zeliznak-1_2022,maros-zeliznak-1_2023,mate-colina-1_2022,matheus-silveira-1_2022,matthew-nicholson-1_2022,matthew-nicholson-1_2023,max-brackmann-1_2022,mihai-carcoana-1_2023,mitchell-saxen-1_2022,mitchell-saxen-1_2023,mo-njie-1_2023,morris-udeze-1_2022,morris-udeze-1_2023,moses-bol-1_2023,moulaye-sissoko-1_2022,moulaye-sissoko-1_2023,mounir-hima-1_2023,moussa-cisse-1_2022,moussa-cisse-1_2023,naheem-mcleod-1_2023,nate-bittle-1_2023,nate-johnson-7_2022,nate-johnson-7_2023,nate-watson-1_2022,nevin-zink-1_2022,nevin-zink-1_2023,nic-lynch-1_2022,nic-lynch-1_2023,nick-muszynski-1_2022,nick-ongenda-1_2022,nick-ongenda-1_2023,nicolas-pavrette-1_2023,noel-brown-1_2022,noel-brown-1_2023,nysier-brooks-1_2022,osinachi-smart-1_2022,oumar-ballo-1_2022,oumar-ballo-1_2023,paul-newman-4_2022,payton-sparks-1_2023,pierre-sow-1_2023,qudus-wahab-1_2022,qudus-wahab-1_2023,quincy-ba

In [75]:
df_c_cs['zach-edey-1_2023'].sort_values(ascending=False).head(15)

Player
zach-edey-1_2023           1.000000
kofi-cockburn-1_2022       0.987862
mitchell-saxen-1_2023      0.977551
christian-koloko-1_2022    0.974362
efe-abogidi-1_2022         0.969672
ryan-kalkbrenner-1_2022    0.967555
hunter-dickinson-1_2022    0.955688
oumar-ballo-1_2023         0.955657
branden-carlson-1_2023     0.946936
hunter-dickinson-1_2023    0.946689
mark-williams-7_2022       0.938871
zach-edey-1_2022           0.938850
nick-muszynski-1_2022      0.935085
jesse-edwards-1_2023       0.929813
clifford-omoruyi-1_2023    0.927468
Name: zach-edey-1_2023, dtype: float64

In [11]:
def magnitude(v, cols):
  word = (math.sqrt(sum(v[n].iloc[0]*v[n].iloc[0] for n in cols)))
  return word

def cosine(v1, v2, cols):
  dot_product = sum([v1[i].iloc[0]*v2[i].iloc[0] for i in cols])
  return(dot_product/(magnitude(v1, cols)*magnitude(v2, cols)))

#print(cosine(c, d, c.columns))
#matrix = pd.DataFrame()


In [12]:
df_std = df_std.drop(['Pos'], axis=1)

In [13]:

a_list = ['zach-edey-1_2022', 'zach-edey-1_2023', 'hunter-dickinson-1_2022', 'hunter-dickinson-1_2023', 
          'jake-stephens-1_2022', 'tre-mitchell-1_2022', 'tre-mitchell-1_2023']

b_list = list(df_std['Player_Year'].unique())
#print(type(a_list))
matrix = {'id1': [], 'id2': [], 'Similarity': []}
for id1 in a_list:
  print(id1)
  #print(id1, type(id1), )
  for id2 in b_list:
    if id1 == id2: continue
    v1 = df_std[df_std['Player_Year']==id1]
    v1 = v1.drop('Player_Year', axis=1)
    v2 = df_std[df_std['Player_Year']==id2]
    v2 = v2.drop('Player_Year', axis=1)
    matrix['id1'].append(id1)
    matrix['id2'].append(id2)
    #print(v1.columns)
    matrix['Similarity'].append(cosine(v1,v2, list(v1.columns)))
    #v1.columns
    #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

sim_matrix = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

sim_matrix


#matrix

zach-edey-1_2022
zach-edey-1_2023
hunter-dickinson-1_2022
hunter-dickinson-1_2023
jake-stephens-1_2022
tre-mitchell-1_2022
tre-mitchell-1_2023


,id1,id2,Similarity
0,zach-edey-1_2022,keegan-murray-1_2022,0.824580
1,zach-edey-1_2022,darius-mcghee-1_2022,0.364520
2,zach-edey-1_2022,darius-mcghee-1_2023,0.380475
3,zach-edey-1_2022,daylen-kountz-1_2022,0.591199
4,zach-edey-1_2022,daylen-kountz-1_2023,0.413871
...,...,...,...
56394,tre-mitchell-1_2023,jerry-higginsiii-1_2022,-0.255071
56395,tre-mitchell-1_2023,devin-carter-2_2022,-0.421098
56396,tre-mitchell-1_2023,devin-carter-2_2023,-0.725137
56397,tre-mitchell-1_2023,mannie-harris-1_2023,-0.432821


In [14]:
sim_matrix[sim_matrix['id1']=='zach-edey-1_2022'].sort_values('Similarity', ascending=False).head(5)

,id1,id2,Similarity
176,zach-edey-1_2022,kofi-cockburn-1_2022,0.962017
7,zach-edey-1_2022,zach-edey-1_2023,0.957756
770,zach-edey-1_2022,colin-castleton-1_2022,0.951927
6692,zach-edey-1_2022,tolu-smith-1_2023,0.940547
6797,zach-edey-1_2022,nfaly-dante-1_2023,0.937940


In [ ]:
from scipy import spatial
# a_list = list(df_c['Player_Year'].unique())
# b_list = a_list
a_list = ['zach-edey-1_2022', 'zach-edey-1_2023', 'hunter-dickinson-1_2022', 'hunter-dickinson-1_2023']
 #         'jake-stephens-1_2022', 'tre-mitchell-1_2022', 'tre-mitchell-1_2023']

b_list = list(df_c['Player_Year'].unique())
#print(type(a_list))
matrix = {'id1': [], 'id2': [], 'Similarity': []}
for id1 in a_list:
  print(id1)
  #print(id1, type(id1), )
  for id2 in b_list:
    if id1 == id2: continue
    v1 = df_c[df_c['Player_Year']==id1]
    v1 = v1.drop('Player_Year', axis=1)
    v2 = df_c[df_c['Player_Year']==id2]
    v2 = v2.drop('Player_Year', axis=1)
    matrix['id1'].append(id1)
    matrix['id2'].append(id2)
    #matrix['Similarity'].append(1-spatial.distance.cosine(v1.iloc[0], v2.iloc[0]))
    #v1.columns
    #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

sim_matrix_c = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

sim_matrix_c

In [17]:
#df_c = df_std[df_std['Pos']=='C']
#df_c = df_c.drop(['Pos'], axis=1)
df_c_col = list(df_c['Player_Year'])
df_c = df_c.set_index('Player_Year')

In [19]:
df2

,zach-edey-1_2022,zach-edey-1_2023,hunter-dickinson-1_2022,hunter-dickinson-1_2023,jake-stephens-1_2022,jake-stephens-1_2023,kofi-cockburn-1_2022,ahsan-asadullah-1_2022,ahsan-asadullah-1_2023,morris-udeze-1_2022,...,yusuf-jihad-1_2022,yusuf-jihad-1_2023,joe-hedstrom-1_2022,joe-hedstrom-1_2023,zeke-richards-1_2022,zeke-richards-1_2023,grant-asman-1_2022,grant-asman-1_2023,ludgy-debaut-1_2022,ludgy-debaut-1_2023
Temp,,,,,,,,,,,,,,,,,,,,,
zach-edey-1_2022,1.000000,0.957756,0.899990,0.886203,0.794896,0.864522,0.962017,0.766709,0.383357,0.825052,...,0.069285,-0.355053,-0.059171,0.264781,0.368341,0.123550,0.104875,0.220446,0.052619,0.118861
zach-edey-1_2023,0.957756,1.000000,0.924135,0.939474,0.837739,0.862509,0.946686,0.700554,0.336843,0.767564,...,-0.002862,-0.414245,-0.160509,0.193763,0.333606,0.015092,0.056397,0.202789,0.059795,0.076729
hunter-dickinson-1_2022,0.899990,0.924135,1.000000,0.966640,0.869014,0.858247,0.887623,0.717715,0.298416,0.693131,...,-0.124412,-0.311001,-0.098703,0.175420,0.228096,-0.017442,-0.018206,0.249488,0.051091,-0.054319
hunter-dickinson-1_2023,0.886203,0.939474,0.966640,1.000000,0.852399,0.843356,0.894001,0.658615,0.283669,0.703266,...,-0.128733,-0.363222,-0.118904,0.129837,0.222810,-0.082503,-0.008826,0.272840,0.062861,-0.028166
jake-stephens-1_2022,0.794896,0.837739,0.869014,0.852399,1.000000,0.959781,0.742450,0.666091,0.214858,0.473271,...,-0.258231,-0.364159,-0.234403,0.246128,0.199320,-0.126838,0.075670,0.348952,-0.055413,-0.165039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zeke-richards-1_2023,0.123550,0.015092,-0.017442,-0.082503,-0.126838,-0.108709,0.074866,0.018004,0.224729,0.466756,...,0.457399,0.145847,0.299582,0.482796,0.678155,1.000000,-0.026613,-0.096379,0.417543,0.681204
grant-asman-1_2022,0.104875,0.056397,-0.018206,-0.008826,0.075670,0.236686,0.013929,0.297101,0.440567,-0.010113,...,-0.077800,-0.241226,0.285857,-0.060038,-0.036139,-0.026613,1.000000,0.426895,-0.069668,0.072926
grant-asman-1_2023,0.220446,0.202789,0.249488,0.272840,0.348952,0.385291,0.062333,0.296422,0.234905,0.049069,...,0.119146,-0.062878,0.057921,0.289399,0.091059,-0.096379,0.426895,1.000000,0.236773,0.081609


In [20]:
df2['zach-edey-1_2022'].sort_values(ascending=False).head()

Temp
zach-edey-1_2022        1.000000
kofi-cockburn-1_2022    0.962017
zach-edey-1_2023        0.957756
nfaly-dante-1_2023      0.937940
oumar-ballo-1_2023      0.908891
Name: zach-edey-1_2022, dtype: float64

In [ ]:
#sim_matrix_c.to_csv("Similarity_Matrix_Centers.csv")

In [ ]:
sim_matrix_c[sim_matrix_c['id1']=='zach-edey-1_2023'].sort_values('Similarity', ascending=False).head(5)

#sim_matrix_c[sim_matrix_c['id1']=='hunter-dickinson-1_2022'].sort_values('Similarity', ascending=False).head(5)

#sim_matrix_c[sim_matrix_c['id1']=='zeke-richards-1_2022'].sort_values('Similarity', ascending=False).head(5)

,id1,id2,Similarity
305,zach-edey-1_2023,zach-edey-1_2022,0.956708
310,zach-edey-1_2023,kofi-cockburn-1_2022,0.944014
307,zach-edey-1_2023,hunter-dickinson-1_2023,0.939045
593,zach-edey-1_2023,oumar-ballo-1_2023,0.935382
314,zach-edey-1_2023,morris-udeze-1_2023,0.928507


In [ ]:
from scipy import spatial
df_f = df_f.sort_values('Player_Year')
a_list = list(df_f['Player_Year'].unique())
b_list = a_list

#print(type(a_list))
matrix = {'id1': [], 'id2': [], 'Similarity': []}
for id1 in a_list:
  print(id1)
  #print(id1, type(id1), )
  for id2 in b_list:
    if id1 == id2: continue
    v1 = df_f[df_f['Player_Year']==id1]
    v1 = v1.drop('Player_Year', axis=1)
    v2 = df_f[df_f['Player_Year']==id2]
    v2 = v2.drop('Player_Year', axis=1)
    matrix['id1'].append(id1)
    matrix['id2'].append(id2)
    matrix['Similarity'].append(1-spatial.distance.cosine(v1.iloc[0], v2.iloc[0]))
    #v1.columns
    #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

sim_matrix_f = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

sim_matrix_f

aaron-murphy-1_2022
aaron-reddish-1_2022
aaron-reddish-1_2023
aaron-wheeler-2_2022
abayomi-iyiola-1_2022
abayomi-iyiola-1_2023
abdou-ndiaye-1_2022
abdou-samb-1_2023
abdou-tsimbila-1_2022
abdou-tsimbila-1_2023
abdul-ado-1_2022
abdul-momoh-1_2023
abe-johnson-1_2023
abou-ousmane-1_2022
abou-ousmane-1_2023
abu-kigab-1_2022
adam-larson-1_2023
adam-thistlewood-1_2022
adama-sanogo-1_2022
adama-sanogo-1_2023
adem-bona-1_2023
ademide-badmus-1_2023
aguek-arop-1_2022
aguek-arop-1_2023
ahjany-lee-1_2023
ahmard-harvey-1_2023
ahmir-langlais-1_2022
ahmir-langlais-1_2023
aidan-igiehon-1_2022
aidan-igiehon-1_2023
aidan-shaw-1_2023
airion-simmons-1_2022
airion-simmons-1_2023
aj-allenspach-1_2022
aj-casey-1_2023
aj-clayton-1_2022
aj-clayton-1_2023
aj-edu-1_2023
aj-george-1_2023
aj-griffin-1_2022
aj-taylor-1_2022
aj-wilson-2_2022
ajang-aguek-1_2022
ajiri-johnson-1_2022
ajiri-johnson-1_2023
akeem-odusipe-1_2022
akok-akok-1_2022
akok-akok-1_2023
akol-arop-1_2022
akol-arop-1_2023
alanzo-frink-1_2022
alden-ap

,id1,id2,Similarity
0,aaron-murphy-1_2022,aaron-reddish-1_2022,0.225010
1,aaron-murphy-1_2022,aaron-reddish-1_2023,0.006931
2,aaron-murphy-1_2022,aaron-wheeler-2_2022,-0.292554
3,aaron-murphy-1_2022,abayomi-iyiola-1_2022,0.485996
4,aaron-murphy-1_2022,abayomi-iyiola-1_2023,0.575343
...,...,...,...
5815327,zurabi-zhgenti-1_2022,zinou-bedri-1_2023,0.640234
5815328,zurabi-zhgenti-1_2022,zion-griffin-1_2022,-0.273897
5815329,zurabi-zhgenti-1_2022,zion-griffin-1_2023,0.059453
5815330,zurabi-zhgenti-1_2022,zion-tordoff-1_2022,0.449062


In [ ]:
sim_matrix_f.to_csv("Similarity_Matrix_Forwards.csv")

In [ ]:
#sim_matrix_f[sim_matrix_f['id1']=='drew-timme-1_2022'].sort_values('Similarity', ascending=False).head(5)

sim_matrix_f[sim_matrix_f['id1']=='keegan-murray-1_2022'].sort_values('Similarity', ascending=False).head(5)

sim_matrix_f[sim_matrix_f['id1']=='keve-aluma-1_2022'].sort_values('Similarity', ascending=False).head(5)

,id1,id2,Similarity
3339254,keve-aluma-1_2022,adama-sanogo-1_2023,0.980305
3339406,keve-aluma-1_2022,azuolas-tubelis-1_2023,0.972410
3341115,keve-aluma-1_2022,paolo-banchero-1_2022,0.961324
3339801,keve-aluma-1_2022,deandre-williams-1_2023,0.960356
3339890,keve-aluma-1_2022,drew-timme-1_2023,0.960145


In [ ]:
from scipy import spatial
df_g = df_g.sort_values('Player_Year')
a_list = list(df_g['Player_Year'].unique())
b_list = a_list

#print(type(a_list))
matrix = {'id1': [], 'id2': [], 'Similarity': []}
for id1 in a_list:
  print(id1)
  #print(id1, type(id1), )
  for id2 in b_list:
    if id1 == id2: continue
    v1 = df_g[df_g['Player_Year']==id1]
    v1 = v1.drop('Player_Year', axis=1)
    v2 = df_g[df_g['Player_Year']==id2]
    v2 = v2.drop('Player_Year', axis=1)
    matrix['id1'].append(id1)
    matrix['id2'].append(id2)
    matrix['Similarity'].append(1-spatial.distance.cosine(v1.iloc[0], v2.iloc[0]))
    #v1.columns
    #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

sim_matrix_g = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

sim_matrix_g

aamer-muhammad-1_2023
aanen-moody-1_2022
aanen-moody-1_2023
aaron-cash-1_2022
aaron-cash-1_2023
aaron-clarke-1_2022
aaron-cook-2_2022
aaron-cooley-1_2022
aaron-cooley-1_2023
aaron-deloney-1_2022
aaron-deloney-1_2023
aaron-duhart-1_2022
aaron-estrada-1_2022
aaron-estrada-1_2023
aaron-gray-2_2023
aaron-lemon-warren-1_2023
aaron-scott-5_2022
aaron-scott-5_2023
aaron-thompson-3_2022
aaryn-rai-1_2022
abdoulaye-thiam-1_2023
abdul-mohamed-1_2022
aboubacar-traore-1_2022
aboubacar-traore-1_2023
ac-curry-1_2023
achile-spadone-1_2023
adam-flagler-1_2022
adam-flagler-1_2023
adam-hess-2_2023
adam-kunkel-1_2022
adam-kunkel-1_2023
adam-miller-3_2023
adam-seiko-1_2022
adam-seiko-1_2023
adam-silas-1_2023
adama-bal-1_2023
adante-holiman-1_2023
ade-adebayo-1_2022
ade-adebayo-1_2023
adefolalrin-adetogun-1_2022
adefolalrin-adetogun-1_2023
adetokunbo-bakare-1_2023
adham-eleeda-1_2022
adonis-arms-1_2022
adou-thiero-1_2023
adrian-balwinjr-1_2022
adrian-balwinjr-1_2023
adrian-delph-1_2022
adrian-martin-2_2022


,id1,id2,Similarity
0,aamer-muhammad-1_2023,aanen-moody-1_2022,0.320461
1,aamer-muhammad-1_2023,aanen-moody-1_2023,0.518100
2,aamer-muhammad-1_2023,aaron-cash-1_2022,-0.698760
3,aamer-muhammad-1_2023,aaron-cash-1_2023,0.013353
4,aamer-muhammad-1_2023,aaron-clarke-1_2022,0.561260
...,...,...,...
16986757,zytarious-mortle-1_2023,zion-russell-1_2022,0.339291
16986758,zytarious-mortle-1_2023,zion-styles-1_2023,0.600846
16986759,zytarious-mortle-1_2023,zion-williams-2_2022,0.456867
16986760,zytarious-mortle-1_2023,zyon-pullin-1_2022,-0.074825


In [ ]:
sim_matrix_g.to_csv("Similarity_Matrix_Guards.csv")

In [ ]:
#sim_matrix['True'] = 1-sim_matrix['Similarity']
sim_matrix = pd.concat([sim_matrix_c, sim_matrix_f, sim_matrix_g])
sim_matrix

,id1,id2,Similarity
0,zach-edey-1_2022,zach-edey-1_2023,0.956708
1,zach-edey-1_2022,hunter-dickinson-1_2022,0.900140
2,zach-edey-1_2022,hunter-dickinson-1_2023,0.885337
3,zach-edey-1_2022,jake-stephens-1_2022,0.795056
4,zach-edey-1_2022,jake-stephens-1_2023,0.863639
...,...,...,...
16986757,zytarious-mortle-1_2023,zion-russell-1_2022,0.339291
16986758,zytarious-mortle-1_2023,zion-styles-1_2023,0.600846
16986759,zytarious-mortle-1_2023,zion-williams-2_2022,0.456867
16986760,zytarious-mortle-1_2023,zyon-pullin-1_2022,-0.074825


In [ ]:
sim_matrix.to_csv("Full_Similarity_Matrix.csv")

In [ ]:
sim_matrix[sim_matrix['id1']=='james-bishop-2_2020'].sort_values('Similarity', ascending=False).head(5)

sim_matrix[sim_matrix['id1']=='brendan-adams-1_2022'].sort_values('Similarity', ascending=False).head(5)

sim_matrix[sim_matrix['id1']=='keegan-murray-1_2022'].sort_values('Similarity', ascending=False).head(5)

,id1,id2,Similarity
151,keegan-murray-1_2022,david-roddy-1_2022,0.953149
211,keegan-murray-1_2022,keve-aluma-1_2022,0.951684
116,keegan-murray-1_2022,kj-williams-1_2023,0.949189
173,keegan-murray-1_2022,kris-murray-1_2023,0.948218
107,keegan-murray-1_2022,deandre-williams-1_2023,0.921589


In [ ]:
# from scipy import spatial
# a_list = list(df_std['Player_Year'].unique())
# a_list = a_list[4800:5300]
# #print(type(a_list))
# matrix = {'id1': [], 'id2': [], 'Similarity': []}
# for id1 in a_list:
#   print(id1)
#   #print(id1, type(id1), )
#   for id2 in a_list:
#     if id1 == id2: continue
#     v1 = df_std[df_std['Player_Year']==id1]
#     v1 = v1.drop('Player_Year', axis=1)
#     v2 = df_std[df_std['Player_Year']==id2]
#     v2 = v2.drop('Player_Year', axis=1)
#     matrix['id1'].append(id1)
#     matrix['id2'].append(id2)
#     matrix['Similarity'].append(1-spatial.distance.cosine(v1.iloc[0], v2.iloc[0]))
#     #v1.columns
#     #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

# sim_matrix2 = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

# sim_matrix2

In [ ]:
# a = list(df_std['Player_Year'].unique())
# a = a[0:4800]
# a[4799]

'kamani-johnson-1_2022'